# Train Graph NN on Call Mentions

In [1]:
# Settings
model_name = 'exclude_previous_rating_model_3'
inductive_or_transductive = 'Transductive'

In [2]:
# Packages
from Graph_NN_Functions import *

## Load Learnable Network Data

In [3]:
# Load feature and class data
feature_and_class_df = pd.read_excel('../../../Data/Learnable Network/feature_and_class_df.xlsx')
feature_and_class_df

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,...,Ovrst,Undrst,PN,SW,AP,OU,tone,num_q_by_len,train_test_80_20,node
0,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,364,131,5.518519,15.261905,2.661290,2.778626,3.188264,0.003822,train,1142
1,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,465,152,5.348485,15.934783,3.296482,3.059211,3.681858,0.002766,train,2685
2,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,AA,2015-05-28,AA,2014-05-27,2015-05-28,...,468,151,3.927711,8.113636,2.841346,3.099338,1.307366,0.004628,train,3985
3,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,AA,2015-08-25,AA,2015-05-28,2015-08-25,...,415,135,5.250000,9.142857,2.640187,3.074074,2.025933,0.003861,train,4681
4,AAPL,2015-10-01,2015-07-21,AA,2015-08-25,AA,2016-05-20,AA,2015-06-02,2016-05-20,...,449,148,4.209877,10.442857,2.579909,3.033784,1.815531,0.003915,train,2690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,ZTS,2015-10-01,2015-08-04,BBB,2015-01-30,BBB,2015-11-03,BBB,2014-01-31,2015-11-03,...,298,148,3.611650,15.634615,2.911215,2.013514,1.744657,0.001458,train,4150
4825,ZTS,2016-01-01,2015-11-03,BBB,2015-11-03,BBB,2016-01-22,BBB,2015-01-30,2016-01-22,...,395,222,3.766917,15.848101,2.791667,1.779279,1.596294,0.003859,train,833
4826,ZTS,2016-04-01,2016-02-16,BBB,2016-01-22,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,...,469,217,3.565517,17.506849,2.926829,2.161290,2.287146,0.003928,train,3236
4827,ZTS,2016-07-01,2016-05-04,BBB,2016-01-22,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,...,449,215,3.572650,15.235294,3.023715,2.088372,1.739992,0.003182,train,871


In [4]:
src_dst_df = pd.read_excel('../../../Data/Learnable Network/src_dst_df.xlsx')
src_dst_df

,src,dst
0,1142,1166
1,1855,1166
2,857,1166
3,2356,1166
4,4783,1166
...,...,...
12917,3106,2848
12918,4601,1271
12919,2315,1623
12920,1402,3009


In [5]:
node_to_int = pd.read_excel('../../../Data/Learnable Network/node_to_int.xlsx')
# convert to dictionary
node_to_int = node_to_int.set_index('node').to_dict()['node_int']
node_to_int

{'BDC : 2015-10-01': 0,
 'EQIX : 2012-07-01': 1,
 'LNT : 2012-07-01': 2,
 'SM : 2014-07-01': 3,
 'SPR : 2015-01-01': 4,
 'FCX : 2015-01-01': 5,
 'AGCO : 2013-07-01': 6,
 'AWR : 2016-01-01': 7,
 'GEL : 2014-07-01': 8,
 'GLDD : 2015-07-01': 9,
 'HAS : 2016-01-01': 10,
 'IPG : 2011-10-01': 11,
 'LAMR : 2013-01-01': 12,
 'DHI : 2015-04-01': 13,
 'MLM : 2015-04-01': 14,
 'NWL : 2011-10-01': 15,
 'HAS : 2014-07-01': 16,
 'HON : 2012-07-01': 17,
 'VRSK : 2016-10-01': 18,
 'DOV : 2012-01-01': 19,
 'AVT : 2016-04-01': 20,
 'BMY : 2012-07-01': 21,
 'CLMT : 2012-01-01': 22,
 'HPE : 2016-10-01': 23,
 'MCHP : 2016-04-01': 24,
 'FLR : 2016-01-01': 25,
 'HON : 2013-01-01': 26,
 'MWA : 2015-04-01': 27,
 'PM : 2013-10-01': 28,
 'ALB : 2016-01-01': 29,
 'MRK : 2014-01-01': 30,
 'BIIB : 2016-04-01': 31,
 'PNM : 2016-01-01': 32,
 'OC : 2015-07-01': 33,
 'MEOH : 2016-04-01': 34,
 'ALGT : 2016-07-01': 35,
 'GIS : 2014-04-01': 36,
 'AMKR : 2013-04-01': 37,
 'SLCA : 2014-10-01': 38,
 'EMN : 2016-10-01': 39,
 

In [6]:
# Load column names
numeric_feature_columns, cat_feature_columns, target_column, custom_mapping = get_column_names_and_mapping(model_name)

In [7]:
# Prepare matrices
X_train_scaled, X_test_scaled, y_train, y_test, feature_names, train_ticker_by_fixed_quarter_date, test_ticker_by_fixed_quarter_date = prepare_matrices(feature_and_class_df, numeric_feature_columns, cat_feature_columns, target_column, custom_mapping)

feature names: 
['num__EBIT' 'num__common_plus_preferred_stock' 'num__workingCapital'
 'num__Ratio_A' 'num__Ratio_B' 'num__Ratio_C' 'num__Ratio_D'
 'num__Ratio_E' 'num__Positiv' 'num__Negativ' 'num__Strong' 'num__Weak'
 'num__Active' 'num__Passive' 'num__Ovrst' 'num__Undrst'
 'num__cashAndCashEquivalents' 'num__shortTermInvestments'
 'num__cashAndShortTermInvestments' 'num__netReceivables'
 'num__inventory_balance_sheet' 'num__otherCurrentAssets'
 'num__totalCurrentAssets' 'num__propertyPlantEquipmentNet'
 'num__goodwill' 'num__intangibleAssets'
 'num__goodwillAndIntangibleAssets' 'num__longTermInvestments'
 'num__taxAssets' 'num__otherNonCurrentAssets'
 'num__totalNonCurrentAssets' 'num__otherAssets' 'num__totalAssets'
 'num__accountPayables' 'num__shortTermDebt' 'num__taxPayables'
 'num__deferredRevenue' 'num__otherCurrentLiabilities'
 'num__totalCurrentLiabilities' 'num__longTermDebt'
 'num__deferredRevenueNonCurrent' 'num__deferredTaxLiabilitiesNonCurrent'
 'num__otherNonCurrentLia

In [8]:
# Assemble back into dataframes

# Train and val
train_and_val_df = pd.DataFrame(X_train_scaled, columns=feature_names)
print(train_and_val_df)
# Add y_train
train_and_val_df[target_column] = y_train.reset_index(drop=True)
# Add ticker by fixed quarter date
train_and_val_df = pd.concat([train_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), train_and_val_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
train_and_val_df = train_and_val_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
train_and_val_df = train_and_val_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

# Test
test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
# Add y_test
test_df[target_column] = y_test.reset_index(drop=True)
# Add ticker by fixed quarter date
test_df = pd.concat([test_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), test_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
test_df = test_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
test_df = test_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

print('finalized dfs')
print(train_and_val_df)
print(test_df)
print('missing values of target_column in train_and_val_df or test_df?')
print(train_and_val_df[target_column].isnull().sum() > 0)
print(test_df[target_column].isnull().sum() > 0)

      num__EBIT  num__common_plus_preferred_stock  num__workingCapital  \
0     -0.535373                         -0.358207            -0.275469   
1     -0.533847                         -0.357010            -0.279550   
2     -0.511245                         -0.356119            -0.278396   
3     -0.521557                         -0.354906            -0.278551   
4     -0.528805                         -0.353937            -0.279407   
...         ...                               ...                  ...   
3844  -0.090070                         -0.375685             0.146378   
3845  -0.008860                         -0.375685             0.158135   
3846  -0.299142                         -0.375685             0.221085   
3847   0.098268                         -0.375685             0.244354   
3848   0.055072                         -0.375685             0.279870   

      num__Ratio_A  num__Ratio_B  num__Ratio_C  num__Ratio_D  num__Ratio_E  \
0         1.389023     -0.249585 

## Run the Model

In [9]:
run_model(train_and_val_df = train_and_val_df,
         test_df = test_df,
         src_dst_df = src_dst_df,
         model_dir = '../../../Output/Modelling/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '/',
         prediction_file_path = '../../../Data/Predictions/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '_predictions.xlsx',
         target_column = target_column,
         custom_mapping = custom_mapping,
         node_to_int = node_to_int,
         n_hidden = 128,
         n_layers = 2,
         dropout = 0.4,
         weight_decay = 1e-2,
         n_epochs = 300,
         lr = 0.01,
         aggregator_type = "pool",
         inductive = (inductive_or_transductive == 'Inductive'))

Further slice the train dataset into train and validation datasets.
The training data has shape: (3079, 155).
The validation data has shape: (770, 155).
The test data has shape: (980, 155).
Generate train, validation, and test masks.
sum of train mask
tensor(3079)
sum of val mask
tensor(770)
sum of test mask
tensor(980)
Number of nodes = 4829
Number of features for each node = 153
Number of classes = 9.
Initializing Model
Initialized Model
NodeClassification(
  (gconv_model): GraphSAGEModel(
    (layers): ModuleList(
      (0): SAGEConv(
        (feat_drop): Dropout(p=0.4, inplace=False)
        (fc_pool): Linear(in_features=153, out_features=153, bias=True)
        (fc_neigh): Linear(in_features=153, out_features=128, bias=False)
        (fc_self): Linear(in_features=153, out_features=128, bias=True)
      )
      (1): SAGEConv(
        (feat_drop): Dropout(p=0.4, inplace=False)
        (fc_pool): Linear(in_features=128, out_features=128, bias=True)
        (fc_neigh): Linear(in_featu